In [1]:
#%run ./methods.ipynb

In [2]:
#df = pd.read_pickle('./data/fitbitflexwirelessactivitysleepwristbandblack.pkl')

In [3]:
#%run ./tfidf.ipynb

In [4]:
# add and remove stopwords
stopwords = ['iphone', 'fitbit', 'alta', 'fit bit', 'hr', 'sony', 'mdr', 'mdr7506', 'product', 'device', 'thing', 'love','daughter','son','husband','wife','mother','father','star','pair','thing','product','mom','dad','day','week','month','year','minute','second']
no_stopwords = ['no', 'not']
stopword_list = nltk.corpus.stopwords.words('english')
for stopword in stopwords:
    stopword_list.append(stopword)
for no_stopword in no_stopwords:
    stopword_list.remove(no_stopword)

In [5]:
merged_tokens = list(itertools.chain(*tokens))

bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(merged_tokens)
trigram_finder = nltk.collocations.TrigramCollocationFinder.from_words(merged_tokens)

bigram_finder.apply_freq_filter(5)
trigram_finder.apply_freq_filter(5)

trigram_scores = {}
for key, value in trigram_finder.ngram_fd.items():
    freq = trigram_finder.score_ngram(trigram_measures.raw_freq,key[0],key[1],key[2])
    pmi =  trigram_finder.score_ngram(trigram_measures.pmi,key[0],key[1],key[2])
    trigram_scores[' '.join(key)]=freq*pmi*sum([len(key[0]),len(key[1]),len(key[2])])
    
bigram_scores = {}
for key, value in bigram_finder.ngram_fd.items():
    freq = bigram_finder.score_ngram(bigram_measures.raw_freq,key[0],key[1])
    pmi =  bigram_finder.score_ngram(bigram_measures.pmi,key[0],key[1])
    bigram_scores[' '.join(key)]=freq*pmi*sum([len(key[0]),len(key[1])])
    

In [6]:
trigrams = sorted(trigram_scores, key=trigram_scores.get, reverse=True)[:len(df.noun_phrases)]
bigrams = sorted(bigram_scores, key=bigram_scores.get, reverse=True)[:len(df.noun_phrases)]

In [7]:
# extract noun chunks from sentences and clear
reviews_trigrams = []
reviews_bigrams = []
trigram_entities = []
bigram_entities = []
for review in df.preprocessed:
    
    review = preprocess(
            review,
            isolate_special_characters=True,
            remove_special_characters=True,
            remove_extra_whitespace=True,
            remove_stopwords=True,
            remove_numbers=True)
    
    tmp_trigrams = []
    tmp_trigram_entities = []
    tmp_bigram_entities = []
    for trigram in trigrams:
        if trigram in review:
            tmp_trigrams.append(trigram)
            trigram = extract_entity(trigram)
            tmp_trigram_entities.append(trigram)
    reviews_trigrams.append(tmp_trigrams)
    trigram_entities.append(tmp_trigram_entities)
    
    tmp_bigrams = []
    for bigram in bigrams:
        if bigram in review:
            tmp_bigrams.append(bigram)
            bigram = extract_entity(bigram)
            tmp_bigram_entities.append(bigram)
    reviews_bigrams.append(tmp_bigrams)
    bigram_entities.append(tmp_bigram_entities)


In [8]:
bigram_scores_ = []
for i,review in enumerate(bigram_entities):
    tmp_topic = []
    for j,ph in enumerate(review):
        try:
            tmp_topic.append((sum([d[preprocess(word, lemmatize=True)]for word in ph.split()])/len(ph.split()))*len(reviews_bigrams[i][j].split()))
        except:
            tmp_topic.append(None)
    bigram_scores_.append(tmp_topic)

In [9]:
trigram_scores_ = []
for i,review in enumerate(trigram_entities):
    tmp_topic = []
    for j,ph in enumerate(review):
        try:
            tmp_topic.append((sum([d[preprocess(word, lemmatize=True)]for word in ph.split()])/len(ph.split()))*len(reviews_trigrams[i][j].split()))
        except:
            tmp_topic.append(None)
    trigram_scores_.append(tmp_topic)

In [10]:
df["bigrams"] = reviews_bigrams
df["bigrams_scores"] = bigram_scores_
df["trigrams"] = reviews_trigrams
df["trigrams_scores"] = trigram_scores_
df["bigram_entities"] = bigram_entities
df["trigram_entities"] = trigram_entities
#df.to_pickle('./data/fitbitflexwirelessactivitysleepwristbandblack.pkl')